In [ ]:
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from micrograd.engine import Value
from micrograd.nn import MLP

In [ ]:
np.random.seed(1337)
random.seed(1337)

**Build a Dataset**

In [ ]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=128, noise=0.1)
# make y be -1 or 1
y = y*2 - 1 

# visualize in 2D
plt.figure(figsize=(5,5))
plt.scatter(X[:,0], X[:,1], c=y, s=20, cmap='jet')

**Define a Model**

In [ ]:
# initialize a model 
model = MLP(2, [16, 16, 1])
print(f"number of parameters = {len(model.parameters())}")

In [ ]:
# define loss function
def loss(batch_size: int | None = None) -> tuple[Value, float]:
    # inline DataLoader
    if batch_size is None:
        Xb, yb = X, y
    else:
        ri = np.random.permutation(X.shape[0])[:batch_size]
        Xb, yb = X[ri], y[ri]
    inputs = [list(map(Value, xrow)) for xrow in Xb]
    
    # forward the model to get scores
    scores = list(map(model, inputs))
    
    # SVM "max-margin" loss (tanh instead of relu)
    losses = [(1 + -yi*scorei).tanh() for yi, scorei in zip(yb, scores)]
    data_loss = sum(losses) * (1.0 / len(losses))
    
    # L2 regularization
    alpha = 1e-4
    reg_loss = alpha * sum((p*p for p in model.parameters()))
    
    # total loss
    total_loss = data_loss + reg_loss
    assert isinstance(total_loss, Value)
    
    # also get accuracy
    accuracy = [(yi > 0) == (scorei.data > 0) for yi, scorei in zip(yb, scores)]
    return total_loss, sum(accuracy) / len(accuracy)

total_loss, acc = loss()
print(f"total loss = {total_loss.data}, accuracy = {acc}")

**Optimization**

In [ ]:
N_EPOCHS = 32

# optimization
for k in range(N_EPOCHS):
    # forward
    total_loss, acc = loss()
    
    # backward
    model.zero_grad()
    total_loss.backward()
    
    # update (sgd) with learning rate decay
    learning_rate = 1.0 - 0.9*k/100
    for p in model.parameters():
        p.data -= learning_rate * p.grad
    
    if k % 2 == 0:
        print(f"step {k} loss {total_loss.data}, accuracy {acc*100}%")

In [ ]:
total_loss, acc = loss()
print(f"loss {total_loss.data}, accuracy {acc*100}%")

**Visualize Decision Boundary**

In [ ]:
h = 0.25
x_min, x_max = X[:, 0].min() - 1, X[:, 0].max() + 1
y_min, y_max = X[:, 1].min() - 1, X[:, 1].max() + 1
xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                     np.arange(y_min, y_max, h))
Xmesh = np.c_[xx.ravel(), yy.ravel()]
inputs = [list(map(Value, xrow)) for xrow in Xmesh]
scores = list(map(model, inputs))
Z = np.array([s.data > 0 for s in scores])
Z = Z.reshape(xx.shape)

fig = plt.figure()
plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral, alpha=0.8)
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.Spectral)
plt.xlim(xx.min(), xx.max())
plt.ylim(yy.min(), yy.max())